This notebook is simply a demo of self implemented tf.keras EfficientNet_V2 since lots of people start using it in this competition. 
This notebook mostly adapted from [Alien's 2class notebook](https://www.kaggle.com/h053473666/siim-covid19-efnb7-train-fold0-5-2class), so please take a look at his notebook as well.

In [ ]:
!pip install -U git+https://github.com/GdoongMathew/EfficientNetV2 --no-deps

In [ ]:
import os

import tensorflow as tf
from tensorflow.keras import mixed_precision

# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_global_policy(policy)
import efficientnetv2
from efficientnetv2.utils import DENSE_KERNEL_INITIALIZER, CONV_KERNEL_INITIALIZER
import pandas as pd
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import GroupKFold
import random
import numpy as np

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
        tpu = None
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy, tpu


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        img = tf.image.decode_image(file_bytes, channels=3, expand_animations=False)

#         if ext == 'png':
#             img = tf.image.decode_png(file_bytes, channels=3)
#         elif ext in ['jpg', 'jpeg']:
#             img = tf.image.decode_jpeg(file_bytes, channels=3)
#         else:
#             raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True, final_size=(256, 256, 3)):
        
    def augment(img):
        ########### Add Augmentations here: ###########
        img = tf.image.random_flip_left_right(img)
#         img = tf.image.random_flip_up_down(img)
        random_number = random.randint(0, 2)
        if random_number == 1:    
            img = tf.image.adjust_brightness(img, 0.2)
        if random_number == 2:
            img = tf.image.adjust_brightness(img, -0.2)
        ###############################################
        
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

def build_keras_augmenter():
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.06, fill_mode='constant'),
        tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.06, 
                                                                     width_factor=0.06, 
                                                                     fill_mode='constant'),
        tf.keras.layers.experimental.preprocessing.RandomZoom((-0.4, 0.2), (-0.4, 0.2), fill_mode='constant'),

    ])
    return data_augmentation


def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None, keras_aug_fn=None,
                  augment=True, repeat=True, shuffle=1024, drop_remainder=True,
                  final_size=(256, 256, 3),
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(with_labels=labels is not None, final_size=final_size)
        keras_aug_fn = build_keras_augmenter()
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.batch(bsize, drop_remainder=drop_remainder)
    if labels is not None:
        dset = dset.map(lambda x, y: (keras_aug_fn(x, training=True), y), num_parallel_calls=AUTO) if augment else dset
    else:
        dset = dset.map(lambda x: keras_aug_fn(x, training=True), num_parallel_calls=AUTO) if augment else dset
    dset = dset.prefetch(AUTO)
    
    return dset

In [ ]:
COMPETITION_NAME = "siimcovid19-512-img-png-600-study-png"
strategy, _ = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16
GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)

In [ ]:
load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
df = pd.read_csv('../input/siim-cov19-csv-2class/train.csv')
label_cols = df.columns[4]

In [ ]:
gkf  = GroupKFold(n_splits = 5)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(df, groups = df.StudyInstanceUID.tolist())):
    df.loc[val_idx, 'fold'] = fold
    
image_size = 512

In [ ]:
i = 0
tf.keras.backend.clear_session()

valid_paths = GCS_DS_PATH + '/image/' + df[df['fold'] == i]['id'] + '.png' #"/train/"
train_paths = GCS_DS_PATH + '/image/' + df[df['fold'] != i]['id'] + '.png' #"/train/" 
valid_labels = df[df['fold'] == i][label_cols].values
train_labels = df[df['fold'] != i][label_cols].values
decoder = build_decoder(with_labels=True, target_size=(image_size, image_size), ext='png')

train_dataset = build_dataset(
    train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder,
    augment=True, final_size=(image_size, image_size, 3)
)

valid_dataset = build_dataset(
    valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder,
    repeat=False, shuffle=False, augment=False, final_size=(image_size, image_size, 3)
)

try:
    n_labels = train_labels.shape[1]
except:
    n_labels = 1


In [ ]:
l2_norm = 2e-5
dropout_rate = [0.25, 0.25]

with strategy.scope():
    input_x = tf.keras.layers.Input(shape=(None, None, 3))
    effnet = efficientnetv2.EfficientNetV2_L(weights='imagenet21k-ft1k',
                                             include_top=False,
                                             input_tensor=input_x)

    x = effnet(input_x, training=True)
    x = tf.keras.layers.Dropout(dropout_rate[0])(x)
    output_x = tf.keras.layers.Dense(1,
                                     activation='sigmoid',
                                     kernel_initializer=DENSE_KERNEL_INITIALIZER,
                                     dtype=tf.float32, name='final_dense')(x)

    # blocks_59
    branch_x = effnet.get_layer('normal_block6l_add').output
    branch_x = tf.keras.layers.Dropout(dropout_rate[1], name="aux_dropout")(branch_x)
    branch_x = tf.keras.layers.Conv2D(64, 3, 
                                      kernel_regularizer=tf.keras.regularizers.L2(l2_norm),
                                      kernel_initializer=CONV_KERNEL_INITIALIZER)(branch_x)
    branch_x = tf.keras.layers.BatchNormalization()(branch_x)
    branch_x = tf.keras.layers.Activation('relu')(branch_x)
    branch_x = tf.keras.layers.GlobalAveragePooling2D()(branch_x)
    branch_x = tf.keras.layers.Dense(1, activation='sigmoid',
                                     kernel_initializer=DENSE_KERNEL_INITIALIZER,
                                     dtype=tf.float32, name='b59_dense')(branch_x)

    model = tf.keras.Model(inputs=input_x, outputs=[output_x, branch_x])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
        # loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(multi_label=True)])
model.summary()
steps_per_epoch = train_paths.shape[0] // BATCH_SIZE

save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f'effv2_model_{i}.h5', save_best_only=True, monitor='val_final_dense_loss', mode='min', options=save_locally, verbose=1)
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_final_dense_loss", patience=3, min_lr=5e-7, mode='min', verbose=1, factor=0.4)

#     tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'../logs/effv2_model_{i}', profile_batch=(5, 15))

history = model.fit(
    train_dataset,
    epochs=24,
    verbose=1,
    callbacks=[checkpoint, lr_reducer],
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_dataset)

hist_df = pd.DataFrame(history.history)
hist_df.to_csv(f'history{i}.csv')
del model, effnet